In [ ]:
!unzip /content/Test.zip

In [40]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

In [41]:
suffix = '.pgm'
kaggle_files = os.listdir('../Test/Test')
pgm_kaggle_files = [filename for filename in kaggle_files if filename.endswith(suffix)]

In [42]:
len(pgm_kaggle_files)

8000

In [9]:
kaggle_patches = []
for filename in tqdm(pgm_kaggle_files):
  path = '../Test/Test/' + filename
  with open(path, 'rb') as pgmf:
      image = plt.imread(pgmf)
  kaggle_patches.append(image)

100%|██████████| 8000/8000 [01:34<00:00, 84.57it/s] 


In [43]:
pgm_kaggle_files_id = []

for filename in tqdm(pgm_kaggle_files):
  pgm_kaggle_files_id.append(filename[5:-4])

100%|██████████| 8000/8000 [00:00<00:00, 1793587.34it/s]


In [50]:
# Aquí debe utilizar su modelo
model = joblib.load('../modelos/xgb_1.pkl')

In [51]:
# Aquí debe procesar las imagenes de pgm_kaggle_files
pca_model = joblib.load('../modelos/pca_final_15.pkl')
pgm_kaggle_array = np.stack(kaggle_patches)
kaggle_images_flatten = np.array([im.flatten() for im in tqdm(pgm_kaggle_array, desc='Construyendo X')])
X_pca_kag = pca_model.transform(kaggle_images_flatten)


Construyendo X: 100%|██████████| 8000/8000 [00:00<00:00, 252577.62it/s]

In [52]:
X_pca_kag.shape

(8000, 15)

In [53]:
# Predicción
y_kag = model.predict(X_pca_kag)

In [54]:
# Predicción + ID
y_kag_dic = {pgm_kaggle_files_id[i]: y_kag[i] for i in range(len(pgm_kaggle_files_id))}
kaggle_hat = pd.DataFrame(list(y_kag_dic.items()), columns=['id', 'target_feature'])
kaggle_hat['id'] = kaggle_hat['id'].astype(int)
kaggle_hat['target_feature'] = kaggle_hat['target_feature'].astype(int)
kaggle_hat.sort_values(by='id', inplace=True)

In [55]:
kaggle_hat.head()


,id,target_feature
0,0,1
1,1,1
1112,2,1
2223,3,1
3334,4,1


In [56]:
kaggle_hat.value_counts('target_feature')

target_feature
 1    7809
-1     191
Name: count, dtype: int64

In [39]:
# Guardamos en un .csv para subir a kaggle
submission_name = 'submission_name'
kaggle_hat.to_csv('../submissions/test_submission' + '.csv', index=False)